# Basic income would shrink the racial disparity in poverty

In [ ]:
import pandas as pd
import numpy as np
import microdf as mdf
import plotly.express as px
SPM_COLS = ['spm_' + i for i in
            ['id', 'weight', 'povthreshold', 'resources', 'numper']]
raw = pd.read_csv('https://github.com/MaxGhenis/datarepo/raw/master/pppub20.csv.gz', 
                  usecols = ['PRDTRACE', 'MARSUPWT', "AGI"] +
                  [i.upper() for i in SPM_COLS])

In [ ]:
person = raw.copy(deep=True)
person.columns = person.columns.str.lower()
person['weight'] = person.marsupwt / 100
person.spm_weight /= 100
person = person.rename(columns={'prdtrace': 'race'})
# Add indicators for white only and black only (not considering other races).
person['white'] = person.race == 1
person['black'] = person.race == 2
# Limit to positive AGI.
person['agi_pos'] = np.maximum(person.agi, 0)

In [ ]:
# Need total population to calculate UBI and total AGI for required tax rate.
total_population = person.weight.sum()
total_agi = mdf.weighted_sum(person, "agi_pos", "weight")

In [ ]:
# Sum up AGI for each SPM unit and merge that back to person level.
spm = person.groupby(['spm_id'])[['agi_pos', "white", "black"]].sum()
spm.columns = ['spm_' + i for i in spm.columns]
person = person.merge(spm, on='spm_id')

In [ ]:
pop_in_race_spmu = pd.Series({
    "Black": person[person.spm_black > 0].weight.sum(),
    "White": person[person.spm_white > 0].weight.sum()
})
pop_in_race_spmu

Black    4.653551e+07
White    2.581956e+08
dtype: float64

In [ ]:
person.groupby(['black', 'white']).weight.sum()

black  white
False  False    3.377338e+07
       True     2.484004e+08
True   False    4.309445e+07
Name: weight, dtype: float64

In [ ]:
def pov_gap(df, resources, threshold, weight):
    # df: Should be SPM-unit level.
    gaps = np.maximum(df[threshold] - df[resources], 0)
    return (gaps * df[weight]).sum()

In [ ]:
# Maybe do this?
#person = person[person.spm_resources >= 0]

In [ ]:
# Split up SPM weight between black and white people.
person['spm_weight_black'] = (person.spm_weight * person.spm_black / 
                              person.spm_numper)
person['spm_weight_white'] = (person.spm_weight * person.spm_white / 
                              person.spm_numper)

In [ ]:
def pov(race, monthly_ubi):
    # Total cost and associated tax rate.
    cost = monthly_ubi * total_population * 12
    tax_rate = cost / total_agi
    # Calculate new tax, UBI and resources per person, but at SPM unit level.
    person['spm_new_tax'] = tax_rate * person.spm_agi_pos
    person['spm_ubi'] = 12 * monthly_ubi * person.spm_numper
    person['new_resources'] = (person.spm_resources - person.spm_new_tax +
                               person.spm_ubi)
    # Based on new resources, calculate 
    person['new_poor'] = person.new_resources < person.spm_povthreshold
    # person['poverty_gap'] = person.spm_povthreshold - person.new_resources
    # tp = target persons
    tp = person.copy(deep=True)

    if race == 'White':
        tp = tp[tp.white]
    if race == 'Black':
        tp = tp[tp.black]

    tp_pop = tp.weight.sum()
    tp_total_poor = (tp.weight * tp.new_poor).sum()
    pov_rate_tot = ((tp_total_poor/tp_pop)*100).round(2)

    # Calculate poverty gap by creating a new SPM unit level dataset.
    tp = tp.drop_duplicates(subset=['spm_id'])
    poverty_gap = pov_gap(tp, "new_resources", "spm_povthreshold",
                          "spm_weight_" + race.lower())
    # poverty_gap = (tp.new_poor * tp.spm_weight * tp.poverty_gap).sum()
    poverty_gap_per_capita = (poverty_gap / pop_in_race_spmu[race]).round(2)

    return pd.Series([poverty_gap_per_capita, pov_rate_tot])

In [ ]:
summary = mdf.cartesian_product({'race': ['White', 'Black'], 
                                 'monthly_ubi': np.arange(0, 1001, 50)})

In [ ]:
def pov_row(row):
    return pov(row.race, row.monthly_ubi)

summary[['poverty_gap_per_capita', 'poverty_rate']] = summary.apply(pov_row, axis=1)

In [ ]:
summary

,race,monthly_ubi,poverty_gap_per_capita,poverty_rate
0,White,0,426.23,10.48
1,White,50,378.96,9.24
2,White,100,337.21,8.14
3,White,150,300.81,7.08
4,White,200,269.30,6.23
5,White,250,241.69,5.47
6,White,300,217.69,4.81
7,White,350,196.60,4.30
8,White,400,178.18,3.84
9,White,450,161.98,3.38


In [ ]:
def line_graph(df, x, y, color, title, xaxis_title, 
               yaxis_title, color_discrete_map, yaxis_ticksuffix,
               yaxis_tickprefix):
    """Style for line graphs.
    
    Arguments
    df: DataFrame with data to be plotted.
    x: The string representing the column in df that holds the new spending in billions.
    y: The string representing the column in df that holds the poverty rate.
    color: The string representing the UBI type.
    xaxis_title: The string represnting the xaxis-title.
    yaxis_title: The string representing the yaxis-title.
    
    Returns
    Nothing. Shows the plot.
    """
    fig = px.line(df, x=x, y=y, color=color,
                  color_discrete_map=color_discrete_map)
    fig.update_layout(
        title=title,
        xaxis_title=xaxis_title,
        yaxis_title=yaxis_title,
        yaxis_ticksuffix=yaxis_ticksuffix,
        yaxis_tickprefix=yaxis_tickprefix,
        font=dict(family='Roboto'),
        hovermode='x',
        xaxis_tickprefix='$',
        plot_bgcolor='white',
        legend_title_text='',
        height=600,
        width=1000,
        
    )

    fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.9
    ))

    fig.update_traces(mode='markers+lines', hovertemplate=None)

    fig.show()

In [ ]:
DARK_BLUE = '#1565C0'
GRAY = '#808080'

line_graph(df=summary, x='monthly_ubi', 
           y='poverty_rate', color='race',
           title='Black and white poverty rate by monthly UBI',
           xaxis_title='Monthly UBI',
           yaxis_title='SPM poverty rate',                  
           color_discrete_map={'White': GRAY,
                        'Black': DARK_BLUE},
           yaxis_ticksuffix='%',
           yaxis_tickprefix='')

In [ ]:
line_graph(df=summary, x='monthly_ubi', 
           y='poverty_gap_per_capita', color='race',
           title='Black and white poverty gap per capita by monthly UBI',
           xaxis_title='Monthly UBI',
           yaxis_title='Poverty gap per capita',                  
           color_discrete_map={'White': GRAY,
                               'Black': DARK_BLUE},
           yaxis_ticksuffix='',
           yaxis_tickprefix='$')

Multiplicative delta

In [ ]:
wide = summary.pivot_table(['poverty_rate', 'poverty_gap_per_capita'], 'monthly_ubi', 'race')
wide.columns = ['pg_black', 'pg_white', 'pr_black', 'pr_white']
wide['pg_ratio'] = wide.pg_black / wide.pg_white
wide['pr_ratio'] = wide.pr_black / wide.pr_white
wide.reset_index(inplace=True)

In [ ]:
ratios = wide.melt(id_vars='monthly_ubi', value_vars=['pr_ratio', 'pg_ratio'])
line_graph(df=ratios, x='monthly_ubi', 
           y='value', color='variable',
           title='Ratio of black and white estimate at UBI level',
           xaxis_title='Monthly UBI',
           yaxis_title='Ratio',                  
           color_discrete_map={'pr_ratio': GRAY,
                               'pg_ratio': 'green'},
           yaxis_ticksuffix='',
           yaxis_tickprefix='')